In [1]:
from bs4 import BeautifulSoup
import requests
import numpy as np
from time import sleep
import json
from tqdm import tqdm
import os
import warnings
warnings.filterwarnings('ignore')

# Tratamiento de datos
# -----------------------------------------------------------------------
import pandas as pd


# Importar librerías para automatización de navegadores web con Selenium
# -----------------------------------------------------------------------
from selenium import webdriver  # Selenium es una herramienta para automatizar la interacción con navegadores web.
from webdriver_manager.chrome import ChromeDriverManager  # ChromeDriverManager gestiona la instalación del controlador de Chrome.
from selenium.webdriver.common.keys import Keys  # Keys es útil para simular eventos de teclado en Selenium.
from selenium.webdriver.support.ui import Select  # Select se utiliza para interactuar con elementos <select> en páginas web.
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException # Excepciones comunes de selenium que nos podemos encontrar 

In [2]:
from src import support_funciones as sf

ModuleNotFoundError: No module named 'src'

In [2]:
def esperar_y_hacer_click_xpath(drivers, ruta_xpath):
    """
    Espera hasta que un elemento esté presente en la página y realiza un clic en él.

    Args:
        drivers (WebDriver): El objeto del navegador (WebDriver) que está siendo utilizado para controlar la página.
        ruta_xpath (str): El XPath del elemento que se está buscando en la página.

    Returns:
        WebElement: El botón o elemento sobre el cual se hizo clic. Retorna `None` si el elemento no se encuentra después de 5 intentos.
    """
    
    boton = None
    intentos = 0
    while True and intentos <= 5:
        try:
            boton = WebDriverWait(drivers, 10).until(EC.presence_of_element_located(("xpath", ruta_xpath)))
            sleep(2)
            boton.click()
            break
        except Exception as e:
            intentos += 1
            print(f"Intento {intentos}: No se ha podido encontrar el botón. Error: {e}")
            continue
    
    return boton

In [3]:
def esperar_y_hacer_click_css(drivers, ruta_css):
    """
    Espera hasta que un elemento esté presente en la página y realiza un clic en él.

    Args:
        drivers (WebDriver): El objeto del navegador (WebDriver) que está siendo utilizado para controlar la página.
        ruta_css (str): El css selector del elemento que se está buscando en la página.

    Returns:
        WebElement: El botón o elemento sobre el cual se hizo clic. Retorna `None` si el elemento no se encuentra después de 5 intentos.
    """
    
    boton = None
    intentos = 0
    while True and intentos <= 5:
        try:
            boton = WebDriverWait(drivers, 10).until(EC.presence_of_element_located(("css selector", ruta_css)))
            sleep(2)
            boton.click()
            break
        except Exception as e:
            intentos += 1
            print(f"Intento {intentos}: No se ha podido encontrar el botón. Error: {e}")
            continue
    
    return boton

In [4]:
driver = webdriver.Chrome()
url="https://super.facua.org/"
driver.get(url)
driver.maximize_window()
sleep(1)
# Aceptamos las cookies cookies
esperar_y_hacer_click_xpath(driver, '//*[@id="rcc-confirm-button"]')
driver.execute_script("window.scrollTo(0, 200)")

diccionario_de_urls = dict.fromkeys(["mercadona", "carrefour", "eroski", "dia", "hipercor", "alcampo"], {})

# Iteramos por cada supermercado
for i in range(1,7):
    sleep(1)
    # Hacemos click en cada supermercado
    esperar_y_hacer_click_css(driver, f"body > section:nth-child(4) > div > div.row.gx-4.gx-lg-6.row-cols-2.row-cols-md-2.row-cols-xl-6.justify-content-center > div:nth-child({i}) > div > div.card-footer.p-4.pt-0.border-top-0.bg-transparent > div > a")    # driver.find_element("css selector", f"body > section:nth-child(4) > div > div.row.gx-4.gx-lg-6.row-cols-2.row-cols-md-2.row-cols-xl-6.justify-content-center > div:nth-child({i}) > div > div.card-footer.p-4.pt-0.border-top-0.bg-transparent > div > a").click()
    sleep(2)
    # Extraemos los tipos de producos (aceite de girasol, aceite de oliva o leche) y los almacenamos en una lsita
    aux_elementos=driver.find_elements("css selector", "body > section:nth-child(4) > div > div.row.gx-4.gx-lg-5.row-cols-2.row-cols-md-3.row-cols-xl-4.justify-content-center")
    lista_aux = [elemento.text for elemento in aux_elementos]
    lista_productos = lista_aux[0].split("\n")[::2]
    j=1
    for producto in lista_productos:
        driver.execute_script("window.scrollTo(0, 300)")
        esperar_y_hacer_click_css(driver, f"body > section:nth-child(4) > div > div.row.gx-4.gx-lg-5.row-cols-2.row-cols-md-3.row-cols-xl-4.justify-content-center > div:nth-child({j}) > div > div.card-footer.p-4.pt-0.border-top-0.bg-transparent > div > a")
        sleep(2)
        # Extraemos los link de los productos
        contenido_html = driver.page_source
        sopa = BeautifulSoup(contenido_html, "html.parser")
        productos = sopa.findAll("div", {"class":"card-footer p-4 pt-0 border-top-0 bg-transparent"})
        lista_links = []
        for prod in productos:
            link = prod.find("a", {"class":"btn-unirme btn-verde inline-block inline-block bg-primary border-primary font-semibold rounded-full"}).get("href")
            lista_links.append(link)
        
        #Sacamos la clave correspondiente para el diccionario
        clave = list(diccionario_de_urls.keys())[i-1]
        diccionario_de_urls[clave][producto]=lista_links

        driver.back()
        sleep(1)
        j+=1

    driver.back()
    sleep(2)
    
driver.quit()

 Guardamos el diccionario como un archivo json y lo cargamos

In [5]:
with open("../datos/diccionario_links.json", "w") as archivo:
    json.dump(diccionario_de_urls, archivo)

In [6]:
with open("../datos/diccionario_links.json", "r") as archivo:
    diccionario_de_urls = json.load(archivo)

Creamos un dataframe para cada producto y supermercado

In [7]:
# Iteramos para cada super
for clave, valor in tqdm(diccionario_de_urls.items()):
    #Iteramos para cada producto
    for producto in lista_productos:
        lista_links = diccionario_de_urls[clave][producto]
        df_producto = pd.DataFrame()
        for link in lista_links:
            # Introducimos un try excep porque hay que manejar el hecho de que algunos links no son de productos sino
            # de una categoría
            try:
                url = link
                res = requests.get(url)
                if (res.status_code == 200):
                    sopa = BeautifulSoup(res.content, "html.parser")

                    tabla = sopa.find("table", {"class":"table table-striped table-responsive text-center"})

                    encabezados = [encabezado.getText() for encabezado in tabla.findAll("th")]

                    filas_tabla = []
                    lista_filas = tabla.findAll("tr")
                    for fila in lista_filas:
                        fila_bs= fila.findAll("td")
                        aux=[elemento.text for elemento in fila_bs]
                        filas_tabla.append(aux)
                    
                    df= pd.DataFrame(filas_tabla[1:], columns=encabezados)
                    df["supermercado"] = clave
                    df["categoria_producto"] = producto
                    df["nombre_producto"] = url.split("/")[-2].replace("-"," ")
                    df_producto = pd.concat([df_producto, df])
                else:
                    print(f"Ha habido un error. Código de error: {res.status_code}")
                    break  
             
            except:
                pass

        df_producto.to_csv(f"../datos/df_{clave}_{producto}.csv")

  0%|          | 0/6 [00:00<?, ?it/s]

100%|██████████| 6/6 [11:23<00:00, 113.91s/it]


## Creación y limpieza del dataframe que almacena todos los datos escrapeados

In [8]:
# Cambiamos el directorio de trabajo a la carpeta datos.
os.chdir("../datos")
datos = os.listdir()[:18]
df_supermercados=pd.DataFrame()
# leemos y concatnaos los archivos almacenados en la carpeta datos que contienen la informacion que hemos escrapeado
for archivo_csv in datos:
    df_aux = pd.read_csv(f"../datos/{archivo_csv}")
    df_supermercados = pd.concat([df_supermercados, df_aux])
df_supermercados

,Unnamed: 0,Día,Precio (€),Variación,supermercado,categoria_producto,nombre_producto
0,0,30/06/2024,"5,88",=,alcampo,Aceite de girasol,campomar nature aceite de girasol ecologico ca...
1,1,01/07/2024,"5,88",=,alcampo,Aceite de girasol,campomar nature aceite de girasol ecologico ca...
2,2,12/07/2024,"5,88",=,alcampo,Aceite de girasol,campomar nature aceite de girasol ecologico ca...
3,3,13/07/2024,"5,88",=,alcampo,Aceite de girasol,campomar nature aceite de girasol ecologico ca...
4,4,14/07/2024,"5,88",=,alcampo,Aceite de girasol,campomar nature aceite de girasol ecologico ca...
...,...,...,...,...,...,...,...
18296,39,23/10/2024,"4,73",=,mercadona,Leche,tierra de sabor leche semidesnatada de vaca 6 ...
18297,40,24/10/2024,"4,73",=,mercadona,Leche,tierra de sabor leche semidesnatada de vaca 6 ...
18298,41,25/10/2024,"4,73",=,mercadona,Leche,tierra de sabor leche semidesnatada de vaca 6 ...
18299,42,26/10/2024,"4,73",=,mercadona,Leche,tierra de sabor leche semidesnatada de vaca 6 ...


In [9]:
df_supermercados.drop(columns=["Unnamed: 0"], inplace=True)

In [10]:
df_supermercados = df_supermercados.reindex(columns=['supermercado', 'categoria_producto','nombre_producto', 'Día', 'Precio (€)', 'Variación'])
df_supermercados.head()

,supermercado,categoria_producto,nombre_producto,Día,Precio (€),Variación
0,alcampo,Aceite de girasol,campomar nature aceite de girasol ecologico ca...,30/06/2024,"5,88",=
1,alcampo,Aceite de girasol,campomar nature aceite de girasol ecologico ca...,01/07/2024,"5,88",=
2,alcampo,Aceite de girasol,campomar nature aceite de girasol ecologico ca...,12/07/2024,"5,88",=
3,alcampo,Aceite de girasol,campomar nature aceite de girasol ecologico ca...,13/07/2024,"5,88",=
4,alcampo,Aceite de girasol,campomar nature aceite de girasol ecologico ca...,14/07/2024,"5,88",=


In [11]:
df_supermercados['Variación'] = df_supermercados['Variación'].str.replace('=', '+0,00 (0,00%)')
df_supermercados['Variación'] = df_supermercados['Variación'].str.replace('(','').str.replace('%)','')

In [12]:
# Dividimos la columna variacion en dos, una con la variación neta y otra con el porcentaje
df_supermercados[['cantidad_variacion', 'porcentaje_variacion']] = df_supermercados['Variación'].str.split(' ', expand=True)

In [13]:
# Eliminamos la columna Variación
df_supermercados.drop(columns=["Variación"], inplace=True)

In [14]:
#Renombarmos
df_supermercados.rename(columns={0: 'cantidad_variacion', 1:"porcentaje_variacion"}, inplace=True)

In [15]:
df_supermercados.head()

,supermercado,categoria_producto,nombre_producto,Día,Precio (€),cantidad_variacion,porcentaje_variacion
0,alcampo,Aceite de girasol,campomar nature aceite de girasol ecologico ca...,30/06/2024,"5,88","+0,00","0,00"
1,alcampo,Aceite de girasol,campomar nature aceite de girasol ecologico ca...,01/07/2024,"5,88","+0,00","0,00"
2,alcampo,Aceite de girasol,campomar nature aceite de girasol ecologico ca...,12/07/2024,"5,88","+0,00","0,00"
3,alcampo,Aceite de girasol,campomar nature aceite de girasol ecologico ca...,13/07/2024,"5,88","+0,00","0,00"
4,alcampo,Aceite de girasol,campomar nature aceite de girasol ecologico ca...,14/07/2024,"5,88","+0,00","0,00"


In [16]:
#Pasamos día a tipo fecha

df_supermercados['Día'] = pd.to_datetime(df_supermercados['Día'], format="mixed", dayfirst=True)

In [17]:
# Pasamos a float el porcentje de variación y el precio
df_supermercados['porcentaje_variacion'] = df_supermercados['porcentaje_variacion'].str.replace(',','.').astype(float)
df_supermercados['Precio (€)'] = df_supermercados['Precio (€)'].str.replace(',','.').astype(float)

Ahora intentaremos sacar una subcategoría a partir del nombre

In [18]:
df_supermercados["nombre_producto"]=df_supermercados["nombre_producto"].str.lower()
df_supermercados["categoria_producto"]=df_supermercados["categoria_producto"].str.lower()


In [19]:



# Uso de la función
df_supermercados = asignar_subcategoria(df_supermercados)





In [20]:
# Transformamos en numero real la columna cantidad_variacion
def normalizar_cantidad_variacion(variacion):
    variacion = variacion.replace(',','.')
    if variacion.startswith('+'):
        return float(variacion[1:])
    else:
        return float(variacion[1:]) * -1

In [21]:
df_supermercados['cantidad_variacion'] = df_supermercados['cantidad_variacion'].apply(sf.normalizar_cantidad_variacion)

In [22]:
df_supermercados=df_supermercados.reindex(columns=['supermercado', 'categoria_producto', 'nombre_producto', 'subcategoria', 'Día', 'Precio (€)',
       'cantidad_variacion', 'porcentaje_variacion'])

In [28]:
df_supermercados.head()

,supermercado,categoria_producto,nombre_producto,subcategoria,Día,Precio (€),cantidad_variacion,porcentaje_variacion
0,alcampo,aceite de girasol,campomar nature aceite de girasol ecologico ca...,nan,2024-06-30,5.88,0.0,0.0
1,alcampo,aceite de girasol,campomar nature aceite de girasol ecologico ca...,nan,2024-07-01,5.88,0.0,0.0
2,alcampo,aceite de girasol,campomar nature aceite de girasol ecologico ca...,nan,2024-07-12,5.88,0.0,0.0
3,alcampo,aceite de girasol,campomar nature aceite de girasol ecologico ca...,nan,2024-07-13,5.88,0.0,0.0
4,alcampo,aceite de girasol,campomar nature aceite de girasol ecologico ca...,nan,2024-07-14,5.88,0.0,0.0


In [27]:
df_supermercados.info()

<class 'pandas.core.frame.DataFrame'>
Index: 202602 entries, 0 to 18300
Data columns (total 8 columns):
 #   Column                Non-Null Count   Dtype         
---  ------                --------------   -----         
 0   supermercado          202602 non-null  object        
 1   categoria_producto    202602 non-null  object        
 2   nombre_producto       202602 non-null  object        
 3   subcategoria          202602 non-null  object        
 4   Día                   202602 non-null  datetime64[ns]
 5   Precio (€)            202602 non-null  float64       
 6   cantidad_variacion    202602 non-null  float64       
 7   porcentaje_variacion  202602 non-null  float64       
dtypes: datetime64[ns](1), float64(3), object(4)
memory usage: 13.9+ MB


In [24]:
df_supermercados.to_csv("../datos/df_supermercados.csv")